#***K21-4934 Assignment-2***

#TASK_01

In [12]:
#Built-in libraries for usage of threads and randomization
import random
import threading

#Variable to store points in circle
pts_circle = 0

#For generating random points and counting the points inside the circle
def gen_pts(pts):
    global pts_circle
    count = 0
    for i in range(pts):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x*2 + y*2 <= 1:
            count += 1
    pts_circle += count

#For generating pi value
def gen_pi(thrds, pts):
    global pts_circle
    threads = []
    for i in range(thrds):
        thread = threading.Thread(target=gen_pts, args=(pts,))
        threads.append(thread)
        thread.start() #To start execution of thread
        #Program will pause in the coming loop for a long period as all threads
        #will wait until the last calling thread has finished it's task
    for thread in threads:
        thread.join() #Synchronisation to block the calling thread until joined
                      #thread has finished
    gen_pi = 4 * pts_circle / (pts * thrds)
    print(f"Value of pi(Approximated): {gen_pi}")

#Using 6 threads for 5000000 points and generating pi
gen_pi(6, 5000000)

Value of pi(Approximated): 2.8749532


#TASK_02

In [26]:
#Class to define 5 processes for question requirement
class Processes:
    def __init__(self, pid, burst_time):
        self.pid = pid
        self.burst_time = burst_time
        self.time_left = burst_time
        self.prio = 0
        self.wait_time = 0

    def finish_chk(self):
        return self.time_left <= 0

    def promote(self):
        self.prio -= 1
        self.wait_time = 0

    def demote(self):
        self.prio += 1
        self.wait_time = 0

    def run(self, time):
        self.time_left -= time
        self.prio += 1
        self.wait_time = 0

    def __repr__(self):
        return f"Processes(pid={self.pid}, burst_time={self.burst_time}, prio={self.prio})"

#Scheduler Class for queues and execution criteria
class MLFQ_Scheduler:
    def __init__(self, quant_1, quant_2):
        self.Queue_1 = []
        self.Queue_2 = []
        self.Queue_3 = []
        self.quant_1 = quant_1
        self.quant_2 = quant_2

    def inc_proc(self, process):
        self.Queue_1.append(process)

    def run(self, time_units):
        time_curr = 0
        while time_curr < time_units:
            if not self.Queue_1 and not self.Queue_2 and not self.Queue_3:
                break

            #Queue-1
            if self.Queue_1:
                process = self.Queue_1.pop(0)
                time = min(process.time_left, self.quant_1)
                process.run(time)
                time_curr += time
                if process.finish_chk():
                    continue
                process.wait_time += time
                if process.wait_time >= 3:
                    process.demote()
                self.Queue_2.append(process)
            
            #Queue-2
            elif self.Queue_2:
                process = self.Queue_2.pop(0)
                time = min(process.time_left, self.quant_2)
                process.run(time)
                time_curr += time
                if process.finish_chk():
                    continue
                process.wait_time += time
                if process.wait_time >= 3:
                    process.demote()
                self.Queue_3.append(process)

            #Queue-3
            elif self.Queue_3:
                process = self.Queue_3.pop(0)
                time = process.time_left
                process.run(time)
                time_curr += time
                if process.finish_chk():
                    continue
                process.wait_time += time
                process.promote()
                self.Queue_3.append(process)

    def display_queue(self):
        print(f"Queue 1: {self.Queue_1}")
        print(f"Queue 2: {self.Queue_2}")
        print(f"Queue 3: {self.Queue_3}")

#Created a scheduler object with quantum time 2 for queue 1 and 4 for queue 2
scheduler = MLFQ_Scheduler(2, 4)

#Process created and added to the scheduler
p1 = Processes(1, 6)
p2 = Processes(2, 8)
p3 = Processes(3, 4)
scheduler.inc_proc(p1)
scheduler.inc_proc(p2)
scheduler.inc_proc(p3)

#Run the scheduler for 10 time units
scheduler.run(10)

scheduler.display_queue()

#The output shows that after running the scheduler for 10 time units,
#both p2 and p3 have been demoted to the second queue, and p1 has finished
#execution and has been removed from the queues. Thus, Queue 1 and Queue 3 are 
#empty and Queue 2 has two processes.

Queue 1: []
Queue 2: [Processes(pid=2, burst_time=8, prio=1), Processes(pid=3, burst_time=4, prio=1)]
Queue 3: []
